# Spotify Churn Cohort Analysis
This notebook performs:
- Mock churn dataset load
- Cohort retention analysis
- Churn patterns by liked songs
- Visualizations
- Suggested growth action

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Load dataset
df = pd.read_csv('spotify_churn_mock.csv')
df['signup_date'] = pd.to_datetime(df['signup_date'])
df.head()

In [ ]:
# Add signup week & days since signup
df['signup_week'] = df['signup_date'].dt.isocalendar().week
df['days_since_signup'] = (datetime.now() - df['signup_date']).dt.days

# Active if listening_days covers most of days since signup
df['is_active'] = df.apply(lambda row: 1 if (row['listening_days'] >= row['days_since_signup'] - 7) else 0, axis=1)
df.head()

In [ ]:
# Cohort retention
cohort = df.groupby('signup_week').agg(
    total_users=('user_id', 'count'),
    active_users=('is_active', 'sum')
).reset_index()
cohort['retention_rate'] = cohort['active_users'] / cohort['total_users']
cohort

In [ ]:
# Plot retention
sns.lineplot(data=cohort, x='signup_week', y='retention_rate')
plt.title('Weekly Cohort Retention')
plt.ylabel('Retention Rate')
plt.xlabel('Signup Week')
plt.show()

In [ ]:
# Churn patterns by liked songs
df['liked_songs_group'] = pd.cut(df['liked_songs'], bins=[-1,5,50], labels=['Low','High'])
pattern = df.groupby('liked_songs_group')['churned'].mean().reset_index()
pattern

In [ ]:
# Visualize churn vs liked songs
sns.barplot(data=pattern, x='liked_songs_group', y='churned')
plt.title('Churn Rate by Liked Songs Group')
plt.ylabel('Churn Rate')
plt.xlabel('Liked Songs Group')
plt.show()

### Suggested Action:
- Users with <5 liked songs churn ~2x more.
- Recommend push notification with personalized playlists to encourage more likes and deeper engagement.
- Example: *"Hey Aashish! Discover songs you’ll love — tap to save your next favorite!"*